In [19]:
import nltk
import pandas as pd
# import re
import string
import contractions
import os
from nltk import word_tokenize, sent_tokenize
# from nltk.corpus import stopwords
from nltk.stem import  WordNetLemmatizer
import inflect
# os.chdir('/home/rinkita/courses/NLPSeminar/nlp-quora-kaggle')
# nltk.download('punkt')
# nltk.download('stopwords')
import numpy as np
from tqdm import tqdm
tqdm.pandas()
from collections import defaultdict

import matplotlib.pyplot as plt

In [20]:
paragram_word_embedding_path = "./data/embeddings/paragram_300_sl999/paragram_300_sl999.txt"
PARA_SIZE = 300
embedding_size=64
num_class=2


In [21]:
data = pd.read_csv("./data/train.csv")
x_train=data.question_text
max_document_len=x_train.map(lambda x: len(x.split(' '))).max()


In [22]:
# """Replace contractions in string of text"""
contracted_input = x_train.progress_apply(lambda row:contractions.fix(row))

100%|██████████| 1306122/1306122 [03:47<00:00, 5751.07it/s]


In [23]:
def clean_text(question_text):
    question_text = question_text.translate(str.maketrans('','',string.punctuation))
    question_text = question_text.translate(str.maketrans('','','1234567890'))
    question_text = " ".join(question_text.split())    
    question_text = question_text.lower().split()
    # stop_words = set(stopwords.words('english'))
    # question_text = [w for w in question_text if not w in stop_words and len(w) >= 3]
    question_text = [w for w in question_text if len(w) >= 3]
    question_text = [w.strip('/"“”') for w in question_text ]
    question_text = " ".join(question_text)
    return question_text

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas


In [24]:


def extract_features(question_texts):  
    word2index = {}
    word2count = {}
    index2word = {0: "pad"}
    n_words = 1  # Count SOS and EOS
    for text in question_texts:
        for word in text:             
            if word not in word2index:
                word2index[word] = n_words
                word2count[word] = 1
                index2word[n_words] = word
                n_words += 1
            else:
                word2count[word] += 1
    return word2index,word2count,index2word,n_words

    

#make all the input text of the same size as size max_length input senetnce, padding with word "PAD"(zero padding)
def make_input(sentence):
    sent_len = len(sentence.split(' '))
    if sent_len < max_document_len:
        padded_sentence = sentence + (max_document_len - len(sentence.split(' '))) * " PAD"
    else:
        padded_sentence = sentence

    return padded_sentence,sent_len


#function to return batch of data
def get_sentence_batch(batch_size,data_x,data_y,data_seqlens,input_metadata):
    
    #shuffling and creating training batch data of batch_size
    instance_indices = list(range(len(data_x)))
    np.random.shuffle(instance_indices)
    batch = instance_indices[:batch_size]
    
    #converting sentence to index vector using word2index dictionary
    x = [[input_metadata.word2index[word] for word in data_x[i].lower().split(' ')]for i in batch]
    y = [data_y[i] for i in batch]
    seqlens = [data_seqlens[i] for i in batch]
    return x,y,seqlens



In [25]:
cleaned_text = contracted_input.progress_apply(lambda row:clean_text(row))

word_tokenised = cleaned_text.progress_apply(lambda text:nltk.word_tokenize(text))

lemmatized_words = word_tokenised.progress_apply(lambda word_list:lemmatize_verbs(word_list))

 

100%|██████████| 1306122/1306122 [01:16<00:00, 16987.00it/s]


In [26]:
word2index,word2count,index2word,n_words = extract_features(lemmatized_words)

In [27]:
def get_para(path_to_paragram,word2index_map):
    embedding_weights = {}
    count_all_words = 0
    with open(path_to_paragram,'r', encoding="utf8", errors="ignore") as f:  
        for line in f:
            vals = line.split(' ')
            word = str(vals[0])
            if word in word2index_map:                
                count_all_words += 1                                 
                coefs = np.asarray(vals[1:],dtype='float32')
                coefs /= np.linalg.norm(coefs)
                embedding_weights[word] = coefs
    return embedding_weights

In [28]:

 word2embedding_dict = get_para(paragram_word_embedding_path,word2index)


In [29]:
embedding_matrix = np.zeros((n_words,PARA_SIZE))

out_of_vocabulary = {}

embeddig_available = 0
for word,index  in word2index.items():
    try:        
        word_embedding = word2embedding_dict[word]
        embedding_matrix[index,:] = word_embedding
    except:
        out_of_vocabulary[word] = word2index[word]
        embedding_matrix[index,:] = [0.0]* PARA_SIZE
       



In [30]:
print("vocabulary size is {}".format(n_words))
print("no. of words for which no pre-trained embedding found is {}".format(len(out_of_vocabulary)))
print("no. of words for which pre-trained embedding is available{}".format(n_words-len(out_of_vocabulary)))
print("embedding found for {} percentage of vocab".format(((n_words-len(out_of_vocabulary))/n_words)*100))
# out_of_vocabulary



vocabulary size is 216210
no. of words for which no pre-trained embedding found is 88753
no. of words for which pre-trained embedding is available127457
embedding found for 58.95055732852319 percentage of vocab


In [31]:
#make all the input text of the same size as size max_length input senetnce, padding with word "PAD"(zero padding)
def make_input(sentence):
    sent_len = len(sentence.split(' '))
    if sent_len < max_document_len:
        padded_sentence = sentence + (max_document_len - len(sentence.split(' '))) * " PAD"
    else:
        padded_sentence = sentence

    return padded_sentence,sent_len


#function to return batch of data
def get_sentence_batch(batch_size,data_x,data_y,data_seqlens,input_metadata):
    
    #shuffling and creating training batch data of batch_size
    instance_indices = list(range(len(data_x)))
    np.random.shuffle(instance_indices)
    batch = instance_indices[:batch_size]
    
    #converting sentence to index vector using word2index dictionary
    x = [[input_metadata.word2index[word] for word in data_x[i].lower().split(' ')]for i in batch]
    y = [data_y[i] for i in batch]
    seqlens = [data_seqlens[i] for i in batch]
    return x,y,seqlens